In [1]:
import os
import sys
from itertools import chain
from functools import reduce

import light_curve as lc
import numpy as np
import pandas as pd
from astropy.table import MaskedColumn

from validutils_zwad.read_data import read_data

## Select passbands and min observations on which to calculate features

In [2]:
passbands=('g', 'r',)
min_obs_count = 100

## Load data & filter out sims that DON'T have enough obs in desired bands

In [3]:
def threshold_nobs(pklfilepath, passbands, min_obs_count):
    X_sim, y_sim = read_data(pklfilepath)
    X_sim_reduced = X_sim[reduce(lambda a, b: a & b, (X_sim['mjd_%s' %pb ].map(len) >= min_obs_count for pb in passbands))]
    
    return X_sim_reduced

# Calculate Features!

In [4]:
def replace_magn_with_flux(s):
    if 'magnitude' in s:
        return s.replace('magnitudes', 'fluxes').replace('magnitude', 'flux')
    return f'{s} for flux light curve'


def create_base_features_class(
        magn_extractor,
        flux_extractor,
        passbands=passbands, 
        min_obs_count=min_obs_count,  
        cls_name='BaseFeatures'
    ):
    feature_names = ([f'{name}_magn' for name in magn_extractor.names]
                     + [f'{name}_flux' for name in flux_extractor.names])
    property_names = {band: [f'{name}_{band}'.lower()
                             for name in feature_names]
                      for band in passbands}
    feature_descriptions = (magn_extractor.descriptions 
                            + [replace_magn_with_flux(desc) for desc in flux_extractor.descriptions])
    property_descriptions = {band: [f'{band}-band {desc}'
                             for desc in feature_descriptions]
                      for band in passbands}
    
    features_count = len(feature_names)
    
    return type(
        cls_name,
        (object,),
        {
            '_passbands': passbands,
            'min_obs_count': min_obs_count,
            'magn_extractor': magn_extractor,
            'flux_extractor': flux_extractor,
            'property_names': property_names,
        }
    )


# TODO: add PERIODOGRAM features!
MAGN_EXTRACTOR = lc.Extractor(
    lc.Amplitude(),
    lc.AndersonDarlingNormal(),
    lc.BeyondNStd(1.0),
    lc.BeyondNStd(2.0),
    lc.Cusum(),
    lc.EtaE(),
    lc.InterPercentileRange(0.02),
    lc.InterPercentileRange(0.1),
    lc.InterPercentileRange(0.25),
    lc.Kurtosis(),
    lc.LinearFit(),
    lc.LinearTrend(),
    lc.MagnitudePercentageRatio(0.4, 0.05),
    lc.MagnitudePercentageRatio(0.2, 0.05),
    lc.MaximumSlope(),
    lc.Mean(),
    lc.MedianAbsoluteDeviation(),
    lc.MedianBufferRangePercentage(0.1),
    lc.MedianBufferRangePercentage(0.2),
    lc.PercentAmplitude(),
    lc.PercentDifferenceMagnitudePercentile(0.05),
    lc.PercentDifferenceMagnitudePercentile(0.1),
    lc.Periodogram(
        peaks=5,
        resolution=10.0,
        max_freq_factor=2.0,
        nyquist='average',
        fast=True,
        features=(
            lc.Amplitude(),
            lc.BeyondNStd(2.0),
            lc.BeyondNStd(3.0),
            lc.StandardDeviation(),
        ),
    ),
    lc.ReducedChi2(),
    lc.Skew(),
    lc.StandardDeviation(),
    lc.StetsonK(),
    lc.WeightedMean(),
)


FLUX_EXTRACTOR = lc.Extractor(
    lc.AndersonDarlingNormal(),
    lc.Cusum(),
    lc.EtaE(),
    lc.ExcessVariance(),
    lc.Kurtosis(),
    lc.MeanVariance(),
    lc.ReducedChi2(),
    lc.Skew(),
    lc.StetsonK(),
)

features_list = []

class Features(create_base_features_class(MAGN_EXTRACTOR, FLUX_EXTRACTOR)):

    def run(self):    
        for band, names in self.property_names.items():
        
            for snid, mjd, mag, magerr in zip(X['snid'], 
                                              X['mjd_%s' %band], 
                                              X['mag_%s' %band], 
                                              X['magerr_%s' %band]):

                t = mjd
                m = mag.astype('float64')
                merr = magerr.astype('float64')
                flux = np.power(10.0, -0.4 * m)
                fluxerr = 0.5 * flux * (np.power(10.0, 0.4 * merr) - np.power(10.0, -0.4 * merr))

                magn_features = self.magn_extractor(
                    t,
                    m,
                    merr,
                    sorted=None,
                    fill_value=None,
                )
                flux_features = self.flux_extractor(
                    t,
                    flux,
                    fluxerr,
                    sorted=None,
                    fill_value=None,
                )
                
                # After successfully calculating features, set sim ID and feature values
                features_dict = {}
                features_dict['sim_id'] = int(str(model_num)+str(snid))
                for name, value in zip(names, chain(magn_features, flux_features)):
                    features_dict[name] = value
                features_list.append(features_dict)
                
        return features_list

## Write/save features

In [5]:
def write_features(features_df, output_folder='./output'):
    suffix=f'_sim_{model_num}_{model}'
    os.makedirs(output_folder, exist_ok=True)
    with open(os.path.join(output_folder, 'feature' + suffix + '.name'), 'w+') as outfile:
        names = features_df.columns
        for name in names:
            outfile.write(name + "\n")
        
    oid = np.memmap(os.path.join(output_folder, 'oid' + suffix + '.dat'), mode='w+',
                    dtype=np.uint64, shape=len(features_df))
    oid[:] = features_df.index[:]
    dtype = [(name, np.float32) for name in names]
    feature = np.memmap(os.path.join(output_folder, 'feature' + suffix + '.dat'), mode='w+',
                        dtype=np.float32, shape=(oid.size, len(features_df.columns))) 

    feature[:] = features_df.values
    # Save as csv
    features_df.to_csv(os.path.join(output_folder, 'df' + suffix + '.csv.gz'), compression='gzip')

## Run code to save features as binary files

In [6]:
# TODO: Don't forget the other pkl files!

pklfilepath_list=['./ztf_dr3_sim_data/ZTF_SIMS_ALLMODELS_3RD_RUN/PALEO_ZTF_MODEL90_SNIa-SALT2.pkl.gz',
                  './ztf_dr3_sim_data/ZTF_SIMS_ALLMODELS_3RD_RUN/PALEO_ZTF_MODEL64_TDE.pkl.gz',
                  './ztf_dr3_sim_data/ZTF_SIMS_ALLMODELS_3RD_RUN/PALEO_ZTF_MODEL62_SNIbc-MOSFIT.pkl.gz',
                  './ztf_dr3_sim_data/ZTF_SIMS_ALLMODELS_3RD_RUN/PALEO_ZTF_MODEL60_SLSN-I.pkl.gz',
                  './ztf_dr3_sim_data/ZTF_SIMS_ALLMODELS_3RD_RUN/PALEO_ZTF_MODEL42_SNII-NMF.pkl.gz',
                  ]
model_list = ['SNIa', 'TDE', 'SNIbc', 'SLSN-I', 'SNII-NMF']
model_num_list = [90, 64, 62, 60, 42]

#if __name__== "__main__":

for pklfilepath, model, model_num in zip(pklfilepath_list, model_list, model_num_list):

    X = threshold_nobs(pklfilepath=pklfilepath, 
                  passbands=passbands,
                  min_obs_count=min_obs_count)
    
    calc = Features()
    features_list = calc.run()
    features_df = pd.DataFrame(features_list).set_index('sim_id')
    features_df = features_df.groupby(level=0).sum() # merge rows w/ same sim_id (otherwise one sim_id has r-rows, then g-rows)
    write_features(features_df=features_df, output_folder='./output')

## Load binary feature files to check

In [7]:
oid = np.memmap(f'./output/oid_sim_{model_num}_{model}.dat', mode='r', dtype=np.uint64)
names = open(f'./output/feature_sim_{model_num}_{model}.name').read().split()
dtype = [(name, np.float32) for name in names]
x = np.memmap(f'./output/feature_sim_{model_num}_{model}.dat', mode='r', dtype=np.float32, shape=(oid.size, len(features_df.columns)))

In [8]:
oid

memmap([    604425,    4219068,    4225818, ..., 9039165265, 9039177848,
        9039197505], dtype=uint64)

In [9]:
names

['amplitude_magn_g',
 'anderson_darling_normal_magn_g',
 'beyond_1_std_magn_g',
 'beyond_2_std_magn_g',
 'cusum_magn_g',
 'eta_e_magn_g',
 'inter_percentile_range_2_magn_g',
 'inter_percentile_range_10_magn_g',
 'inter_percentile_range_25_magn_g',
 'kurtosis_magn_g',
 'linear_fit_slope_magn_g',
 'linear_fit_slope_sigma_magn_g',
 'linear_fit_reduced_chi2_magn_g',
 'linear_trend_magn_g',
 'linear_trend_sigma_magn_g',
 'magnitude_percentage_ratio_40_5_magn_g',
 'magnitude_percentage_ratio_20_5_magn_g',
 'maximum_slope_magn_g',
 'mean_magn_g',
 'median_absolute_deviation_magn_g',
 'median_buffer_range_percentage_10_magn_g',
 'median_buffer_range_percentage_20_magn_g',
 'percent_amplitude_magn_g',
 'percent_difference_magnitude_percentile_5_magn_g',
 'percent_difference_magnitude_percentile_10_magn_g',
 'period_0_magn_g',
 'period_s_to_n_0_magn_g',
 'period_1_magn_g',
 'period_s_to_n_1_magn_g',
 'period_2_magn_g',
 'period_s_to_n_2_magn_g',
 'period_3_magn_g',
 'period_s_to_n_3_magn_g',
 'p

In [10]:
x

memmap([[4.1080389e+00, 1.0253984e+00, 2.3015873e-01, ..., 1.3714742e+02,
         3.5859611e+00, 9.1553992e-01],
        [4.0255995e+00, 8.7866855e-01, 3.7500000e-01, ..., 3.3166504e+02,
         1.2867311e+00, 9.0328479e-01],
        [3.4114075e+00, 1.2593510e+00, 3.0097088e-01, ..., 1.8187633e+02,
         9.1150194e-01, 9.1844046e-01],
        ...,
        [4.8525934e+00, 1.3172807e+00, 2.5242719e-01, ..., 1.9413826e+02,
         2.9139967e+00, 8.9492536e-01],
        [4.3420467e+00, 2.3517380e+00, 2.3926380e-01, ..., 1.7191838e+02,
         3.9605386e+00, 9.1974431e-01],
        [3.1006546e+00, 9.2626333e-01, 2.9032257e-01, ..., 1.7850230e+02,
         3.5132902e+00, 9.2539310e-01]], dtype=float32)